In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from datetime import datetime
from scipy.stats import skew  # for some statistics
import seaborn as sns 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy import stats


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

data = train.append(test,sort=False) #Make train set and test set in the same data set


In [ ]:
data.tail()

In [ ]:
quantitative = [f for f in data.columns if data.dtypes[f] != 'object']
quantitative.remove('SalePrice')
quantitative.remove('Id')
qualitative = [f for f in data.columns if data.dtypes[f] == 'object']

In [ ]:
quantitative,len(quantitative)

In [ ]:
qualitative , len(qualitative)

In [ ]:
sns.set_style("whitegrid")
missing = data.isnull().sum()
print(missing)
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

# Missing values 
5 values have missing values more than 50 percent in this case it means lack of the subject not a missing values

In [ ]:
data = data.dropna(axis=1, how='any', thresh = len(data)//2) #Drop columns that more than 50 % of it is data as  NULL values


In [ ]:
columns=data.columns
col_types = data.dtypes
for i in range(len(columns)):
    if col_types[i] == 'object':
        print(f'{columns[i]} mode =', data[columns[i]].mode())
    else:
        print(f'{columns[i]} ===> mean =',data[columns[i]].mean(), "median =", data[columns[i]].median())


In [ ]:
data['BsmtExposure'] = data['BsmtExposure'].fillna('no_basment')
data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])
for i in range(len(columns)):
    if col_types[i] == 'object':
        data[columns[i]] = data[columns[i]].fillna('no')
    elif col_types[i] != 'object' and columns[i]!='SalePrice' :
        mean = data[columns[i]].mean()
        median = data[columns[i]].median()
        if median == 0.0:
            data[columns[i]] = data[columns[i]].fillna(median)
        else:
            data[columns[i]] = data[columns[i]].fillna((mean + median) / 2)


In [ ]:
data
data.isna().sum()

now let's deal with the categorical feature 

In [ ]:
data = pd.get_dummies(data) #Convert string values to integer values
data

In [ ]:
data.dtypes

In [ ]:
covarianceMatrix = data.corr()
listOfFeatures = [i for i in covarianceMatrix]
setOfDroppedFeatures = set() 
for i in range(len(listOfFeatures)) :
    for j in range(i+1,len(listOfFeatures)): #Avoid repetitions 
        feature1=listOfFeatures[i]
        feature2=listOfFeatures[j]
        if abs(covarianceMatrix[feature1][feature2]) > 0.75: #If the correlation between the features is > 0.8
            setOfDroppedFeatures.add(feature1) #Add one of them to the set
#I tried different values of threshold and 0.8 was the one that gave the best results

data = data.drop(setOfDroppedFeatures, axis=1)

In [ ]:
data.shape

In [ ]:
data['SalePrice'] = data['SalePrice'].fillna(data['SalePrice'].mean()) #Replace NULL values with mean values
data['SalePrice'].isna().sum()

In [ ]:
nonCorrelatedWithOutput = [column for column in data if abs(data[column].corr(data["SalePrice"])) < 0.045]
#I tried different values of threshold and 0.045 was the one that gave the best results

data = data.drop(nonCorrelatedWithOutput, axis=1)

In [ ]:
data.shape

removing outliars using the percentile 

In [ ]:
newTrain = data.iloc[:1460]
newTest = data.iloc[1460:]

def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75]) #Get 1st and 3rd quartiles (25% -> 75% of data will be kept)
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5) #Get lower bound
    upper_bound = quartile_3 + (iqr * 1.5) #Get upper bound
    return np.where((ys > upper_bound) | (ys < lower_bound)) #Get outlier values

#Third, we will drop the outlier values from the train set

trainWithoutOutliers = newTrain #We can't change train while running through it

for column in newTrain:
    outlierValuesList = np.ndarray.tolist(outliers_iqr(newTrain[column])[0]) #outliers_iqr() returns an array
    trainWithoutOutliers = newTrain.drop(outlierValuesList) #Drop outlier rows
    
trainWithoutOutliers = newTrain

**normalize the labels**

In [ ]:
X = trainWithoutOutliers.drop("SalePrice", axis=1) #Remove SalePrice column
Y = np.log(trainWithoutOutliers["SalePrice"])


plt.figure(2); plt.title('Normal')
sns.distplot(Y, kde=False, fit=stats.norm)

Now the labels are normally distributed

In [ ]:
reg = LinearRegression().fit(X, Y)


In [ ]:
newTest = newTest.drop("SalePrice", axis=1) #Remove SalePrice column
pred = np.exp(reg.predict(newTest))
print(pred)
#Submit prediction

sub = pd.DataFrame() #Create a new DataFrame for submission
sub['Id'] = test['Id']
sub['SalePrice'] = pred
sub.to_csv("submission.csv", index=False) #Convert DataFrame to .csv file
